In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Step 1: Load the dataset
file_path = r"C:\Users\THARUN\wheather prediction\weather_data_until_nov_2024.csv"
data = pd.read_csv(file_path)

# Step 2: Feature Engineering - Create cyclical features for day and month
data['Day_sin'] = np.sin(2 * np.pi * data['Day'] / 31)
data['Day_cos'] = np.cos(2 * np.pi * data['Day'] / 31)
data['Month_sin'] = np.sin(2 * np.pi * data['Month'] / 12)
data['Month_cos'] = np.cos(2 * np.pi * data['Month'] / 12)

# Step 3: Select features and target
X = data[['Year', 'Day_sin', 'Day_cos', 'Month_sin', 'Month_cos']].values
y = data['Temperature (°C)'].values

# Step 4: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 6: Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Step 7: Define the neural network model
class WeatherPredictionModel(nn.Module):
    def __init__(self, input_size):
        super(WeatherPredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Step 8: Set device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Step 9: Initialize the model and define the optimizer and loss function
input_size = X_train.shape[1]
model = WeatherPredictionModel(input_size).to(device)
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 10: Train the model
num_epochs = 9000
for epoch in range(num_epochs):
    model.train()
    
    # Forward pass
    outputs = model(X_train_tensor.to(device))
    loss = criterion(outputs, y_train_tensor.to(device))
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Step 11: Save the trained model's weights
model_path = 'best_weather_prediction_model.pt'
torch.save(model.state_dict(), model_path)
print(f'Model saved to {model_path}')

# Step 12: Evaluate the model on test data
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor.to(device))
    test_loss = criterion(test_outputs, y_test_tensor.to(device))
    print(f'Test Loss: {test_loss.item():.4f}')


Epoch [100/9000], Loss: 222.7070
Epoch [200/9000], Loss: 46.5168
Epoch [300/9000], Loss: 43.0019
Epoch [400/9000], Loss: 43.5925
Epoch [500/9000], Loss: 41.9933
Epoch [600/9000], Loss: 43.5152
Epoch [700/9000], Loss: 40.1744
Epoch [800/9000], Loss: 43.0922
Epoch [900/9000], Loss: 42.6931
Epoch [1000/9000], Loss: 39.7942
Epoch [1100/9000], Loss: 43.0257
Epoch [1200/9000], Loss: 40.0342
Epoch [1300/9000], Loss: 40.0191
Epoch [1400/9000], Loss: 38.3165
Epoch [1500/9000], Loss: 40.5800
Epoch [1600/9000], Loss: 40.6168
Epoch [1700/9000], Loss: 39.2178
Epoch [1800/9000], Loss: 38.5173
Epoch [1900/9000], Loss: 40.4789
Epoch [2000/9000], Loss: 36.4089
Epoch [2100/9000], Loss: 37.6962
Epoch [2200/9000], Loss: 39.0339
Epoch [2300/9000], Loss: 37.3985
Epoch [2400/9000], Loss: 37.3082
Epoch [2500/9000], Loss: 36.8093
Epoch [2600/9000], Loss: 38.5681
Epoch [2700/9000], Loss: 36.0222
Epoch [2800/9000], Loss: 36.1364
Epoch [2900/9000], Loss: 37.1573
Epoch [3000/9000], Loss: 37.4438
Epoch [3100/9000],

In [ ]:
## Example usage for prediction
year = int(input("Enter the year:"))
month = int(input("Enter the month:"))
day = int(inpur("Enter the date:"))
predicted_temperature = predict_temperature(year,month, day)
print(f'Predicted temperature for {year}-{month}-{day}: {predicted_temperature:.2f}°C')
